In [1]:
# 1. RANDOM FOREST

from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor


In [2]:
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")["RUL_steps"].values
X_val = pd.read_csv("../data/processed/X_val.csv")
y_val = pd.read_csv("../data/processed/y_val.csv")["RUL_steps"].values
train_balanced =  pd.read_csv("../data/processed/train_balanced.csv")
train_df = pd.read_csv("../data/processed/train_df.csv")
val_df = pd.read_csv("../data/processed/val_df.csv")

In [3]:
weights = train_balanced['risk_bin'].map({
    'MUY_CRITICO': 8.0,
    'CRITICO': 5.0, 
    'ALTO_RIESGO': 2.0,
    'BAJO_RIESGO': 1.0
}).values

# 1. RANDOM FOREST (NO necesita escalado)
rf_model = RandomForestRegressor(n_estimators=500, max_depth=10, min_samples_split=10,
                                min_samples_leaf=5, max_features='sqrt', random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train, sample_weight=weights)
rf_pred = rf_model.predict(X_val)

# MÉTRICAS RANDOM FOREST (CRÍTICOS RUL<50)
mask_critica = y_val < 50
rf_mae_total = mean_absolute_error(y_val, rf_pred)
rf_mae_criticos = mean_absolute_error(y_val[mask_critica], rf_pred[mask_critica])
rf_r2_total = r2_score(y_val, rf_pred)

print(f"📊 RANDOM FOREST:")
print(f"MAE total:     {rf_mae_total:.2f}")
print(f"MAE RUL<50:    {rf_mae_criticos:.2f}")
print(f"R² total:      {rf_r2_total:.4f}")
print(f"Nº críticos:   {mask_critica.sum():,} ({mask_critica.mean()*100:.1f}%)")
print()




📊 RANDOM FOREST:
MAE total:     35.71
MAE RUL<50:    12.23
R² total:      -0.3691
Nº críticos:   24,643 (62.1%)

